<a href="https://colab.research.google.com/github/Arvind-55555/MSW-Landfill-Estimation-for-Top-10-Indian-Cities/blob/main/Municipal_Solid_Waste_(MSW)_Landfill_Estimation_for_Top_10_Indian_Cities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd
import numpy as np
import requests
import json
from math import exp, log

In [33]:
# --- 1. Data Fetching Functions ---
def fetch_worldometer_population(city_name):
    """
    Fetches city population from Worldometer API (not always updated).
    """
    try:
        url = "https://www.data.gov.in/search"
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        # Find the table containing population data (inspect the website to find the correct table)
        table = soup.find('table', class_="table table-striped table-bordered")

        # Check if the table is found
        if table is not None:
            # Extract population data from the table using specific tags or attributes
            # The following assumes the population is in the second column (index 1) of the last row
            population_row = table.find_all('tr')[-1]
            population = int(population_row.find_all('td')[1].text.replace(",", ""))
            return population
        else:
            print(f"Table not found for {city_name}. Check the HTML structure.")
            return None  # Return None if the table is not found

    except requests.exceptions.RequestException as e:
        print(f"Error fetching population for {city_name}: {e}")
        return None

def fetch_demographics(city_name):
    """
    Fetches BR, DR for city. Uses a workaround for API challenges.
    """
    try:
        url = f"https://www.data.gov.in/search?"
        response = requests.get(url)
        response.raise_for_status()
        #print(response.text)
        # Use pandas read_html instead of json parsing to parse the demographic table from the web
        dfs = pd.read_html(response.text)
        demographics_df = dfs[0]
        # Assuming the Birth Rate is in the second column (index 1) of the last row
        birth_rate = float(demographics_df.iloc[-1, 1])

        url = f"https://www.data.gov.in/search?"  # Updated URL
        response = requests.get(url)
        response.raise_for_status()
        dfs = pd.read_html(response.text)
        demographics_df = dfs[0]

        # Assuming the Death Rate is in the second column (index 1) of the last row
        death_rate = float(demographics_df.iloc[-1, 1])

        return birth_rate, death_rate

    except requests.exceptions.RequestException as e:
        print(f"Error fetching BR/DR for {city_name}: {e}")
        return None, None

def fetch_landfill_data(city_name):
  """
    Fetches landfill data (very simplified, may require manual data in future)
  """
  # This is a placeholder, replace with a real data source or CSV if needed
  landfill_data = {
    "Mumbai": {"present_capacity_sqkm": 10, "shortage_sqkm": 0.5},
    "Delhi": {"present_capacity_sqkm": 15, "shortage_sqkm": 0.8},
    "Bangalore": {"present_capacity_sqkm": 10, "shortage_sqkm": 0.2},
    "Chennai": {"present_capacity_sqkm": 12, "shortage_sqkm": 0.3},
    "Hyderabad": {"present_capacity_sqkm": 8, "shortage_sqkm": 0.4},
    "Kolkata": {"present_capacity_sqkm": 12, "shortage_sqkm": 0.6},
    "Ahmedabad": {"present_capacity_sqkm": 7, "shortage_sqkm": 0.2},
    "Pune": {"present_capacity_sqkm": 9, "shortage_sqkm": 0.1},
    "Surat": {"present_capacity_sqkm": 6, "shortage_sqkm": 0.3},
    "Jaipur": {"present_capacity_sqkm": 7, "shortage_sqkm": 0.1},
    }
  if city_name in landfill_data:
    return landfill_data[city_name]['present_capacity_sqkm'], landfill_data[city_name]['shortage_sqkm']
  else:
    return None, None

def calculate_population_growth_rate(birth_rate, death_rate):
   """
    Calculates population growth rate based on birth and death rates
   """
   if birth_rate is not None and death_rate is not None:
      return (birth_rate - death_rate) / 100 #convert to percentage
   else:
      return None

In [34]:
# --- 2. Core Calculation Functions ---
def calculate_msw_landfill(population, waste_per_person_per_day_kg, gr):
    """
    Calculates the total landfill area required over 20 years.
    """
    total_msw_kg_over_20_years = 0
    population_projected = population
    for year in range(20):
        daily_msw_tonnes = (population_projected * waste_per_person_per_day_kg) / 1000
        annual_msw_tonnes = daily_msw_tonnes * 365
        total_msw_kg_over_20_years += (annual_msw_tonnes * 1000)
        population_projected = population_projected * (1+gr/100)

    landfill_volume_m3 = total_msw_kg_over_20_years / 600
    total_cell_volume_m3 = landfill_volume_m3 / 0.8  # Account for soil cover
    landfill_area_m2 = total_cell_volume_m3 / 10
    landfill_area_km2 = landfill_area_m2 / 1000000
    return landfill_area_km2, total_msw_kg_over_20_years/1000/1000 #Returns Area in km2 and Total MSW in million tons

In [35]:
# --- 3. Main Execution ---
if __name__ == "__main__":
    top_cities = [
        "Mumbai",
        "Delhi",
        "Bangalore",
        "Chennai",
        "Hyderabad",
        "Kolkata",
        "Ahmedabad",
        "Pune",
        "Surat",
        "Jaipur",
    ]
    waste_per_person_per_day_kg = 0.7 # average
    results = []
    for city in top_cities:
        print(f"Processing {city}...")
        # Fetch Data
        population = fetch_worldometer_population(city)
        birth_rate, death_rate = fetch_demographics(city)
        present_capacity, shortage = fetch_landfill_data(city)
        gr = calculate_population_growth_rate(birth_rate, death_rate)

        # Skip if Data unavailable
        if population is None or birth_rate is None or death_rate is None:
             print(f"Skipping {city} due to data issues...")
             continue

        # Calculations
        landfill_area, msw_tonnes = calculate_msw_landfill(population, waste_per_person_per_day_kg, gr)

        # Append to Result
        results.append({
            "City": city,
            "Population": population,
            "Birth Rate": birth_rate,
            "Death Rate": death_rate,
            "Population GR": gr,
            "Current Landfill Capacity (sqkm)": present_capacity,
            "Landfill Shortage (sqkm)": shortage,
            "Total Landfill Area Needed (km²)": landfill_area,
            "MSW (million tonnes)":msw_tonnes,
        })
    # Convert Results
    df = pd.DataFrame(results)
    print(df)

Processing Mumbai...
Table not found for Mumbai. Check the HTML structure.


<ipython-input-33-314da6757689>:39: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(response.text)


ValueError: No tables found